In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import glob
import os
import json
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
import ast
import joblib

In [4]:
with open('C:/Users/emawe/OneDrive/Documents/C1GamesStarterKit-master/algov1-4-3/replays.json') as json_file:
    json_data = json.load(json_file)

dfs = []
for data in json_data[1:]:

    data = data[1:]

    for item in data:
        for key, value in item.items():
            df = pd.DataFrame({key: [value] if not isinstance(value, list) else [str(value)]})
            dfs.append(df)

df = pd.concat(dfs, axis=0, ignore_index=True)
df = df.reset_index(drop=True)


KeyboardInterrupt: 

In [2]:
max_files = 25

directories = ['C:/Users/emawe/OneDrive/Documents/C1GamesStarterKit-master/scripts/batch_0']

big_dfs = []


def process_file(file):
    print(f"Processing file: {file}")
    with open(file) as f:
        try:
            json_big_data = json.load(f)

            for data in json_big_data[1:]:

                for key, value in data.items():
                    df = pd.DataFrame({key: [value] if not isinstance(value, list) else [str(value)]})
                    big_dfs.append(df)

        except Exception as e:
            print(f"Error processing file {file}: {e}")

file_counter = 0

for directory in directories:
    print(f"Processing directory: {directory}")
    for file in glob.glob(os.path.join(directory, '*.json')):
        process_file(file)
        file_counter += 1
        if file_counter >= max_files:
            break
    if file_counter >= max_files:
        break

big_df = pd.concat(big_dfs, ignore_index=True)
big_df = big_df.reset_index(drop=True)

Processing directory: C:/Users/emawe/OneDrive/Documents/C1GamesStarterKit-master/scripts/batch_0
Processing file: C:/Users/emawe/OneDrive/Documents/C1GamesStarterKit-master/scripts/batch_0\replay_13846434.json
Processing file: C:/Users/emawe/OneDrive/Documents/C1GamesStarterKit-master/scripts/batch_0\replay_13846440.json
Processing file: C:/Users/emawe/OneDrive/Documents/C1GamesStarterKit-master/scripts/batch_0\replay_13846448.json
Processing file: C:/Users/emawe/OneDrive/Documents/C1GamesStarterKit-master/scripts/batch_0\replay_13846462.json
Processing file: C:/Users/emawe/OneDrive/Documents/C1GamesStarterKit-master/scripts/batch_0\replay_13846465.json
Processing file: C:/Users/emawe/OneDrive/Documents/C1GamesStarterKit-master/scripts/batch_0\replay_13846470.json
Processing file: C:/Users/emawe/OneDrive/Documents/C1GamesStarterKit-master/scripts/batch_0\replay_13846472.json
Processing file: C:/Users/emawe/OneDrive/Documents/C1GamesStarterKit-master/scripts/batch_0\replay_13846500.json

In [3]:
non_nan_indices = big_df[~big_df['endStats'].isna()].index.tolist()

df = big_df.drop(non_nan_indices)
df = df.drop(columns=['endStats'])

compressed_df = pd.DataFrame()

for column in df.columns:
    non_nan_values = df[column].dropna().reset_index(drop=True)
    compressed_df[column] = non_nan_values
    
compressed_df.drop(columns=['turnInfo'], inplace=True)
compressed_df.drop(columns=['events'], inplace=True)
print(compressed_df.head())


                                             p2Units               p1Stats  \
0                   [[], [], [], [], [], [], [], []]  [30.0, 40.0, 5.0, 0]   
1  [[[19, 15, 40.0, '29'], [8, 15, 40.0, '30'], [...   [30.0, 1.0, 0.0, 8]   
2  [[[19, 15, 40.0, '29'], [8, 15, 40.0, '30'], [...   [30.0, 1.0, 0.0, 8]   
3  [[[19, 15, 40.0, '29'], [8, 15, 40.0, '30'], [...   [30.0, 1.0, 0.0, 8]   
4  [[[19, 15, 40.0, '29'], [8, 15, 40.0, '30'], [...   [30.0, 1.0, 0.0, 8]   

                                             p1Units               p2Stats  
0                   [[], [], [], [], [], [], [], []]  [30.0, 40.0, 5.0, 0]  
1  [[[2, 12, 40.0, '2'], [3, 11, 40.0, '3'], [4, ...   [30.0, 0.0, 1.0, 5]  
2  [[[2, 12, 40.0, '2'], [3, 11, 40.0, '3'], [4, ...   [30.0, 0.0, 1.0, 5]  
3  [[[2, 12, 40.0, '2'], [3, 11, 40.0, '3'], [4, ...   [30.0, 0.0, 1.0, 5]  
4  [[[2, 12, 40.0, '2'], [3, 11, 40.0, '3'], [4, ...   [30.0, 0.0, 1.0, 5]  


In [6]:
compressed_df['prev_p1Units'] = compressed_df['p1Units'].shift(1)
compressed_df['prev_p2Units'] = compressed_df['p2Units'].shift(1)
compressed_df.dropna(inplace=True)

In [7]:
def encode_nested_lists(nested_list):
    flat_list = [item for sublist in nested_list for item in sublist]
    return flat_list

for col in ['prev_p1Units', 'prev_p2Units']:
    compressed_df[f'encoded_{col}'] = compressed_df[col].apply(encode_nested_lists)

all_values = [item for col in ['encoded_prev_p1Units', 'encoded_prev_p2Units']
              for sublist in compressed_df[col] for item in sublist]
unique_values = sorted(set(all_values))

le = LabelEncoder()
le.fit(unique_values)

def encode_list(lst):
    return le.transform(lst).tolist()


for col in ['encoded_prev_p1Units', 'encoded_prev_p2Units']:
    compressed_df[col] = compressed_df[col].apply(encode_list)


max_len = max(compressed_df[col].apply(len).max() for col in ['encoded_prev_p1Units', 'encoded_prev_p2Units'])

def pad_or_truncate(lst, max_len):
    return lst[:max_len] + [0] * (max_len - len(lst))

for col in ['encoded_prev_p1Units', 'encoded_prev_p2Units']:
    compressed_df[col] = compressed_df[col].apply(lambda x: pad_or_truncate(x, max_len))

X = compressed_df[['encoded_prev_p1Units', 'encoded_prev_p2Units']]
X = np.array(X.apply(lambda row: np.concatenate(row), axis=1).tolist())
X_df = pd.DataFrame(X)
print(X_df.head())

   0     1     2     3     4     5     6     7     8     9     ...  2582  \
0    14    14    15     2     0    14    15     2     0    14  ...     0   
1    14    14    14     6     2     0     5     6     2     0  ...     0   
2    14    14    14     6     2     0     5     6     2     0  ...     0   
3    14    14    14     6     2     0     5     6     2     0  ...     0   
4    14    14    14     6     2     0     5     6     2     0  ...     0   

   2583  2584  2585  2586  2587  2588  2589  2590  2591  
0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0  
3     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0  

[5 rows x 2592 columns]


In [28]:
print(unique_values)

[' ', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'D', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w']


In [8]:
def encode_target_lists(nested_list):
    flat_list = [item for sublist in nested_list for item in sublist]
    return flat_list

compressed_df['encoded_p1Units_target'] = compressed_df['p1Units'].apply(encode_target_lists)
compressed_df['encoded_p2Units_target'] = compressed_df['p2Units'].apply(encode_target_lists)

all_values_target = [item for sublist in compressed_df['encoded_p1Units_target'] for item in sublist] + \
                    [item for sublist in compressed_df['encoded_p2Units_target'] for item in sublist]
unique_values_target = sorted(set(all_values_target))

le_target = LabelEncoder()
le_target.fit(unique_values_target)

def encode_list_target(lst):
    return le_target.transform(lst).tolist()

compressed_df['encoded_p1Units_target'] = compressed_df['encoded_p1Units_target'].apply(encode_list_target)
compressed_df['encoded_p2Units_target'] = compressed_df['encoded_p2Units_target'].apply(encode_list_target)

max_len_p1 = compressed_df['encoded_p1Units_target'].apply(len).max()
max_len_p2 = compressed_df['encoded_p2Units_target'].apply(len).max()

def pad_or_truncate_target(lst, max_len):
    return lst[:max_len] + [0] * (max_len - len(lst))

compressed_df['encoded_p1Units_target'] = compressed_df['encoded_p1Units_target'].apply(lambda x: pad_or_truncate_target(x, max_len_p1))
compressed_df['encoded_p2Units_target'] = compressed_df['encoded_p2Units_target'].apply(lambda x: pad_or_truncate_target(x, max_len_p2))

y = compressed_df[['encoded_p1Units_target', 'encoded_p2Units_target']]
y = np.array(y.apply(lambda row: np.concatenate(row), axis=1).tolist())


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=35, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)
    

RandomForestClassifier(max_depth=10, n_estimators=35, random_state=42)

In [8]:
y_pred = rf_model.predict(X_test)

print(y_pred)

[[14 14 14 ...  0  0  0]
 [14 14 14 ...  0  0  0]
 [14 14 14 ...  0  0  0]
 ...
 [14 14 14 ...  0  0  0]
 [14 14 14 ...  0  0  0]
 [14 14 14 ...  0  0  0]]


In [19]:
print("Dimension of y_test:", y_test.ndim)
print("Dimension of y_pred:", y_pred.ndim)

Dimension of y_test: 2
Dimension of y_pred: 2


In [22]:
y_test_flat = y_test.ravel()
y_pred_flat = y_pred.ravel()

print("Flattened y_test shape:", y_test_flat.shape)
print("Flattened y_pred shape:", y_pred_flat.shape)

accuracy = accuracy_score(y_test_flat, y_pred_flat)
print(f"Accuracy: {accuracy:.2f}")

print(y_pred_flat)

Flattened y_test shape: (3607464,)
Flattened y_pred shape: (3607464,)
Accuracy: 0.95
[14 14 14 ...  0  0  0]


In [10]:
joblib.dump(rf_model, 'rf_model.pki')

['rf_model.pki']